# Generate TimeSeries H5 from Library Profile to import in Web GUI


TVB has the custom notion of "profile". Based on the selected TVB profile, we enable/disable some TVB modules and default features (storage, load defaults).

We make a clear distinction between running TVB under WEB_PROFILE or LIBRARY_PROFILE.

Currently it is not possible to switch in the same code from one TVB profile to another, but it is  possible to share data, as described in the current demo. 

You can find more info about TVB profiles here: http://docs.thevirtualbrain.org/manuals/UserGuide/UserGuide-Shell.html#tvb-profiles

## Step 1

Let us start by running a standard TVB simulation, with the LIBRARY_PROFILE 
(from tvb.simulator.lab import * will set the LIBRARY_PROFILE).

In [1]:
%pylab nbagg
from tvb.simulator.lab import *
from tvb.datatypes import time_series
from tvb.basic.config import settings
import numpy as np

Populating the interactive namespace from numpy and matplotlib
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.epileptor.Epileptor.state_variable_range = Final(field_type=<class 'dict'>, default={'x1': array([-2.,  1.]), 'y1': array([-20.,   2.]), 'z': array([2., 5.]), 'x2': array([-2.,  0.]), 'y2': array([0., 2.]), 'g': array([-1.,  1.])}, required=True)
WARNING  default contains values out of the declared domain. Ex 1.0 
   attribute  tvb.simulator.models.epileptor.Epileptor2D.tt = NArray(label='tt', dtype=float64, default=array([1.]), dim_names=(), ndim=None, required=True)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.epileptor.Epileptor2D.state_variable_range = Final(field_type=<class 'dict'>, default={'x1': array([-2.,  1.]), 'z': array([2., 5.])}, required=True)
WARNING  default contains values out of the declar

WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.stefanescu_jirsa.ReducedSetFitzHughNagumo.state_variable_range = Final(field_type=<class 'dict'>, default={'xi': array([-4.,  4.]), 'eta': array([-3.,  3.]), 'alpha': array([-4.,  4.]), 'beta': array([-3.,  3.])}, required=True)
WARNING  default contains values out of the declared domain. Ex 1.0 
   attribute  tvb.simulator.models.stefanescu_jirsa.ReducedSetHindmarshRose.a = NArray(label=':math:`a`', dtype=float64, default=array([1.]), dim_names=(), ndim=None, required=True)
WARNING  default contains values out of the declared domain. Ex 3.0 
   attribute  tvb.simulator.models.stefanescu_jirsa.ReducedSetHindmarshRose.b = NArray(label=':math:`b`', dtype=float64, default=array([3.]), dim_names=(), ndim=None, required=True)
WARNING  default contains values out of the declared domain. Ex 1.0 
   attribute  tvb.simulator.models.stefanescu_jirsa.ReducedSetHindma

In [2]:
jrm = models.JansenRit(mu=numpy.array([0.]), v0=numpy.array([6.]))
monitor = monitors.TemporalAverage(period=2 ** -2)

# the other aspects of the simulator are standard
sim = simulator.Simulator(
    model=jrm,
    connectivity=connectivity.Connectivity.from_file(),
    coupling=coupling.SigmoidalJansenRit(a=numpy.array([10.0])),
    monitors=(monitor,),
    simulation_length=1e3,
).configure()

# run it
(time_array, data_array), = sim.run()

WARNING  File 'hemispheres' not found in ZIP.


/WORK/TVB/tvb-library/tvb/simulator/coupling.py:365: RuntimeWarning: overflow encountered in exp
  pre = self.cmax / (1.0 + numpy.exp(self.r * (self.midpoint - (x_j[:, 0] - x_j[:, 1]))))
/WORK/TVB/tvb-library/tvb/simulator/models/jansen_rit.py:273: RuntimeWarning: overflow encountered in _numba_dfun_jr
  self.A, self.b, self.B, self.J, self.mu


## Step 2

write the simulation result in a H5 file.

In [3]:
from tvb.core.neocom import h5
from tvb.basic.profile import TvbProfile

TvbProfile.set_profile(TvbProfile.COMMAND_PROFILE)

series_of_time = time_series.TimeSeries(data=data_array, time=time_array, sample_period=monitor.period)
h5.store_complete(series_of_time, ".")

<DataType(None, 78e92a18effe41448917467003b98108, TimeSeriesIndex, tvb.core.entities.model.datatypes.time_series, , None, None, None, None, None, None, None)>

Now you should have a file named "TimeSeriesRegion.......h5" in the current folder where ipython notebook has been launched. 

You can take this "TimeSeriesRegion.h5" file and import it into TVB web GUI, as described here:
http://docs.thevirtualbrain.org/manuals/UserGuide/UserGuide-UI_Project.html#data-structure

After import in TVB web GUI, you will have a new TimeSeriesRegion file in your current project, which can be used with TVB web visualizers.

